In [ ]:
import keras
print(keras.__version__)
#2.3.1
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x  #gpu
except Exception:
  pass
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import cifar100
from keras.utils import np_utils
from google.colab import drive
drive.mount('/content/drive')

Using TensorFlow backend.


2.3.1
`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  #gpu`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from os import listdir
import cv2
X_data = []
y_labels = []
cnt = 0
path = '/content/drive/My Drive/Tri_tue_NTTDK/face_images'
label_to_index = {}
for forder_name in listdir(path):
  label_to_index[forder_name] = cnt
  cnt += 1
print(label_to_index)

{'neutral': 0, 'happy': 1, 'angry': 2, 'sad': 3, 'surprise': 4}


In [ ]:
num_of_images = {'neutral': 0, 'happy': 1, 'angry': 2, 'sad': 3, 'surprise': 4}
for forder_name in listdir(path):
  count = 0
  for file_name in listdir(path + '/' + forder_name):
    path_of_image = path + '/' + forder_name + '/' + file_name
    print(path_of_image)
    image = cv2.imread(path_of_image)
    print(image.shape)
    print(count)
    count += 1
    if count == 700:
      break
    # image = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    image = cv2.resize(image,(224,224))
    image = image/255
    X_data.append(image)
    y_labels.append(label_to_index[forder_name])
  cnt +=1
  num_of_images[forder_name] = count
print(num_of_images)





Streaming output truncated to the last 5000 lines.
/content/drive/My Drive/Tri_tue_NTTDK/face_images/happy/happy_1.1107434065770898.jpg
(244, 200, 3)
190
/content/drive/My Drive/Tri_tue_NTTDK/face_images/happy/happy_0.2617575581784139.jpg
(240, 193, 3)
191
/content/drive/My Drive/Tri_tue_NTTDK/face_images/happy/happy_-0.44641333374903386.jpg
(242, 196, 3)
192
/content/drive/My Drive/Tri_tue_NTTDK/face_images/happy/happy_-0.5542924340152223.jpg
(236, 191, 3)
193
/content/drive/My Drive/Tri_tue_NTTDK/face_images/happy/happy_-1.5113087894031827.jpg
(241, 198, 3)
194
/content/drive/My Drive/Tri_tue_NTTDK/face_images/happy/happy_-0.5717643966069985.jpg
(240, 195, 3)
195
/content/drive/My Drive/Tri_tue_NTTDK/face_images/happy/happy_-0.9203570758188416.jpg
(240, 195, 3)
196
/content/drive/My Drive/Tri_tue_NTTDK/face_images/happy/happy_-0.37679341137678785.jpg
(247, 197, 3)
197
/content/drive/My Drive/Tri_tue_NTTDK/face_images/happy/happy_-0.44894983100934294.jpg
(242, 199, 3)
198
/content/dri

In [ ]:
with open('/content/drive/My Drive/Tri_tue_NTTDK/data_files/x_data_224x224.npy', 'wb') as f:
    np.save(f,X_data)
with open('/content/drive/My Drive/Tri_tue_NTTDK/data_files/y_labels_faces_224_224.npy', 'wb') as f:
    np.save(f,y_labels)

In [ ]:
X_data = np.array(X_data)
y_labels = np.array(y_labels)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data,y_labels, test_size = 0.1)
y_train_processed = np_utils.to_categorical(y_train,5)
y_test_processed = np_utils.to_categorical(y_test,5)
print(X_train.shape, y_train_processed.shape)

(2298, 224, 224, 3) (2298, 5)


#Tranfer learning

In [ ]:
# They made the library, so we install that library to use some models
!pip install git+https://github.com/rcmalli/keras-vggface.git
# check version of keras_vggface
import keras_vggface
# print version
print(keras_vggface.__version__)
from numpy import expand_dims
# from matplotlib import pyplot
import cv2
from PIL import Image
from numpy import asarray
# from mtcnn.mtcnn import MTCNN
# in keras_vggface hace vggface file, so we import (or call) the VGGFace function
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from keras_vggface.utils import decode_predictions
from scipy.spatial.distance import cosine
from keras.layers import  Dense,Dropout, UpSampling2D,GlobalAveragePooling2D,BatchNormalization
from keras.models import Sequential

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-_pwh90j3
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-_pwh90j3
  Created wheel for keras-vggface: filename=keras_vggface-0.6-cp36-none-any.whl size=8311 sha256=9a55f37570ab6460ca158fe1f01e4605dbc6041cd0e9d93da55084d5ac0f608d
  Stored in directory: /tmp/pip-ephem-wheel-cache-5pc_raj2/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface
0.6


In [ ]:
def make_model_tranfer():
  model_extraction = VGGFace(model='resnet50', include_top = False)
  for layer in model_extraction.layers:
    if isinstance(layer,BatchNormalization):
      layer.trainable = True
    else:
      layer.trainable = False
  model = Sequential()
  # model.add(UpSampling2D(size = (7,7)))
  model.add(model_extraction)
  model.add(GlobalAveragePooling2D()) #https://alexisbcook.github.io/assets/global_average_pooling.png
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(BatchNormalization()) #https://forums.fast.ai/uploads/default/original/2X/9/998a1be6463260f731481106756034c42040e256.jpg
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(BatchNormalization())
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.5))
  model.add(BatchNormalization())
  model.add(Dense(5, activation='softmax'))
  return model
model = make_model_tranfer()  # define our model


In [ ]:
# prepare model for fitting (loss, optimizer, etc)
model.compile(
    loss='categorical_crossentropy',  # we train 100-way classification
    optimizer='adam', 
    metrics=['accuracy']  # report accuracy during training
)
import datetime
print(datetime.datetime.now())
history1 = model.fit(
    X_train, y_train_processed,  # prepared data
    batch_size=12,epochs=30)
print(datetime.datetime.now())
model.summary()
model.save_weights('/content/drive/My Drive/Tri_tue_NTTDK/transfer_model/transfer_weights1_grb_28c_20ep.h5')
model.save('/content/drive/My Drive/Tri_tue_NTTDK/transfer_model/tranfer_model1_grb_28c_20ep.h5')
import datetime
print(datetime.datetime.now())
history2 = model.fit(
    X_train, y_train_processed,  # prepared data
    batch_size=24,epochs=20)
print(datetime.datetime.now())
model.summary()
model.save_weights('/content/drive/My Drive/Tri_tue_NTTDK/transfer_model/transfer_weights15__epochs_grb_28c_30ep.h5')
model.save('/content/drive/My Drive/Tri_tue_NTTDK/transfer_model/tranfer_model15_epochs_grb_28c_30ep.h5')

2020-07-03 04:08:11.914137
Epoch 1/30
2298/2298 [==============================] - 41s 18ms/step - loss: 1.7476 - accuracy: 0.3316
Epoch 2/30
2298/2298 [==============================] - 27s 12ms/step - loss: 1.2593 - accuracy: 0.5013
Epoch 3/30
2298/2298 [==============================] - 27s 12ms/step - loss: 0.9951 - accuracy: 0.6166
Epoch 4/30
2298/2298 [==============================] - 27s 12ms/step - loss: 0.8375 - accuracy: 0.6897
Epoch 5/30
2298/2298 [==============================] - 27s 12ms/step - loss: 0.7275 - accuracy: 0.7446
Epoch 6/30
2298/2298 [==============================] - 27s 12ms/step - loss: 0.6412 - accuracy: 0.7724
Epoch 7/30
2298/2298 [==============================] - 27s 12ms/step - loss: 0.6019 - accuracy: 0.8042
Epoch 8/30
2298/2298 [==============================] - 27s 12ms/step - loss: 0.6149 - accuracy: 0.7920
Epoch 9/30
2298/2298 [==============================] - 27s 12ms/step - loss: 0.5559 - accuracy: 0.8198
Epoch 10/30
2298/2298 [==============